## 实验：批处理量

### 实验概要

批处理（Batch Size）对计算机的运算大有利处，可以大幅缩短每张图像的处理时间。这是因为大多数处理数值计算的库都进行了能够高效处理大型数组运算的最优化。并且，在神经网络的运算中，当数据传送成为瓶颈时，批处理可以减轻数据总线的负荷（严格地讲，相对于数据读入，可以将更多的时间用在计算上）。也就是说，批处理一次性计算大型数组要比分开逐步计算各个小型数组速度更快。

与批处理相关联的概念还包括：

- **Sample** (样本)
- **Epoch** 
- **Batch Size** (批处理量)
- **Iterations** (迭代次数)

#### Sample (样本)

样本是单行数据，一个完整的样本，包含了多个输入到算法中的输入向，以及用于与预测进行比较并计算误差的一个输出项。训练数据集包含许多行数据，也可以说包含了很多样本，样本也可以称为：实例，观察值，输入向量或特征向量。

#### Epoch

一个 **Epoch**，指的是整个数据集被前项传递和后向传递到神经网络一次。然而，当我们的数据集太大时，无法一次性把整个数据集输入，因此，需要把一个 **Epoch** 划分成几个小的 **Batches**，分批处理。

另外，只把整个数据集放到神经网络上训练一次是不够的，我们需要把整个数据集在同一个神经网络上训练很多遍。当训练的 **Epoch** 数量从 1 次逐渐增加，神经网络的权重值就会不断的根据训练集结果进行更新，模型对于样本的拟合情况就会从下图中右边的 **欠拟合**，逐渐变成中间的理想状态 —— 如果继续训练，有可能变成左边的 **过拟合** 状态。

![](./img/1_i_lp_hUFyUD_Sq4pLer28g_06.png)

**最佳的 Epoch 应该设置为多少？** 这个问题没有一个最佳答案。不同的数据集的最佳 **Epoch** 大小是不同的。但是，最佳 **Epoch** 和数据的多样性是相关的。例如，我们要做句子分类任务，A 任务是把所有的句子分到 10 个类别中，B 任务是把句子分到 2 个类别中，那么A任务需要的 **Epoch** 数量会大于B任务。

#### Batch Size (批处理量)

之前说过，当我们的数据集太大时，无法一次性把整个数据集输入，因此，需要把一个 **Epoch** 划分成几个小的 Batches，分批处理。所以，**Batch Size** 就是在一个 **Batch** 中取用的训练 **Sample** (样本) 数。

注意，**Batch Size** 和 number of batches 是两个不同的东西。

#### Iterations (迭代次数)

**Iterations** 指的是，需要计算多少个 **batch** 才能完成一个 **Epoch**。这个需要计算的 **batch** 数就是 **Iterations**。

注意，batches 数量 (number of batches) 就是完成 1 个 **Epoch** 的 **Iterations** 数量。

例如，我们的数据集有 2,000 个训练 **Sample** (样本) 。我们可以把这 2,000 个 **Sample** (样本) 划分成 4 个 Batches，每个 **Batch Size**  是 500 个 **Sample** (样本)。当我们完成 4 个**Iterations** 的训练后，我们就完成了 1 个 **Epoch** 的训练。

在这里，**Batch Size** 是 500，**Iterations** 是 4，**Epoch** 是 1，总的 **Sample** (样本) 数量是 2,000 。

### 实验目标

在本实验中，我们尝试通过神经网络进行手写数字图像的分类。我们暂时假设神经网络学习（训练）已经全部结束，我们使用学习到的参数，实现神经网络的 **推理**。这个推理处理过程，也称为神经网络的 **前向传播（Forward Propagation）**。同时，我们将采用基于批处理的代码实现手写数字图像的分类任务，验证模型的分类精确率 (Accuracy)。另外，在这个过程中，我们将使用 pickle，灵活的对我们的数据进行对象化操作。

本实验使用的数据集是 MNIST 手写数字图像集。 MNIST 是机器学习领域最有名的数据集之一，被应用于从简单的实验到发表的论文研究等各种场合。实际上，在阅读图像识别或机器学习的论文时， MNIST 数据集经常作为实验用的数据出现（近几年逐步被 Fashion MNIST 取代）。MNIST 数据集是由 0 到 9 的数字图像构成的，有很多中不同的格式。训练图像有 6 万张，测试图像有 1 万张，这些图像可以用于学习和推理。MNIST 数据集的一般使用方法是，先用训练图像进行学习，再用学习到的模型度量能在多大程度上对测试图像进行正确的分类。另外 MNIST 数据集的格式有很多，在本实验中，我们直接采用原始的 MNIST 数据集格式，MNIST 的图像数据是 `28` 像素 × `28` 像素的灰度图像（`1` 通道 - 灰度通道），各个像素的取值在 0 到 255 之间。每个图像数据都相应地标有标识该图像中的手写数字的 `7`，`2`，`1`等标签。

> 注意：和求解机器学习问题的步骤分成学习（即：**模型训练**）和推理两个阶段进行）一样，使用神经网络解决问题时，也需要首先使用训练数据（学习数据）进行权重参数的学习；进行 **推理**时，使用刚才学习到的参数，对输入数据进行分类。

### 1. 加载依赖库

除了 Numpy 以外，我们还需要导入几个与本地存储进行数据交互操作的库。其中，`os` 用于实现 Python 与系统交互。另外，值得注意的是 `gzip` 和 `pickle`。

#### gizp 库

由于我们的 MNIST 采用的是压缩后的 `.gz` 文件格式，因此，需要使用 `gzip` 将 MNIST 数据集进行解压为图像文件。

#### pickle 库

Pickle 库用于对 Python 对象结构进行序列化和反序列化，也称为展平。序列化，是指将内存中的对象，转换为可以存储在磁盘上或通过网络发送的字节流的过程。稍后，可以通过检索此字符流并将其反序列化，将其转回为 Python 对象。注意，不要将 Pickle 与压缩相混淆。Pickle 是将对象从一种表示形式（随机存取存储器（RAM）中的数据）转换为另一种表示形式（磁盘上的文本）的过程，而后者是用较少的位对数据进行编码的过程，以节省磁盘空间。

因此，当我们使用 `gzip` 将 MNIST 数据集进行解压为图像文件后，通过 `pickle` 把解压到内存中的图片对象序列化，形成字节流格式并存储到磁盘上；以便之后将字节流文件反序列化，作为 Numpy 对象导入到神经网络进行预测。

In [1]:
# 导入os实现与系统交互
import os      
# 导入gzip执行解压缩
import gzip    
# 导入pickle执行序列化
import pickle  
# 导入numpy科学计算库
import numpy as np 

### 2. 指定数据集参数

从 Google 下载回来的 用于 TensorFlow 训练的 MNIST 原始数据集共有四个文件 ——

- **train-images-idx3-ubyte.gz：** MNIST 图像训练集
- **train-labels-idx1-ubyte.gz：** MNIST 图像标签
- **t10k-images-idx3-ubyte.gz：** MNIST 图像测试集
- **t10k-labels-idx1-ubyte.gz：** MNIST 图像测试标签

所有的文件存放在 ./data 目录下。

In [2]:
base_path = os.environ.get("BATH_PATH",'./data/')
data_path = os.path.join(base_path + "lab6/")
result_path = "result/"
os.makedirs(result_path, exist_ok=True)

# 创建一个字典，指定数据集文件路径并赋值给相关变量
key_file = {
    # MNIST 图像训练集
    'train_img':data_path+'train-images-idx3-ubyte.gz',    
    # MNIST 图像标签
    'train_label':data_path+'train-labels-idx1-ubyte.gz',  
    # MNIST 图像测试集
    'test_img':data_path+'t10k-images-idx3-ubyte.gz',       
    #  MNIST 图像测试标签
    'test_label':data_path+'t10k-labels-idx1-ubyte.gz'      
}

# 指定pickle文件存储路径
dataset_dir = os.path.dirname("./data")
save_file = result_path + "mnist.pkl"

# 指定训练集样本数量
train_num = 60000     
# 指定测试集样本数量
test_num = 10000      
# 指定图像是28像素 × 28像素的灰度图像（1通道）
# —— 相对于 RGB 彩色图像则有（3通道）
img_dim = (1, 28, 28) 
# 指定样本尺寸，为一个 1×28×28 = 784 的一维数组
img_size = 784        

### 3. 定义加载图像/标签函数

分别以二进制模式读取 (`rb`) 图像与标签。

读取图像时，通过 `reshape` 方法，将 1×28×28 的二维数组，转换为 784 的一维数组 ——

<img src="./img/1_06.png" width="60%">

In [3]:
def _load_label(file_name):
    file_path = dataset_dir + "/" + file_name
    
    print("Converting " + file_name + " to NumPy Array ...")
    
    # 使用gzip.open函数，以二进制模式读取标签文件，作为对象 f 存储在RAM中。
    with gzip.open(file_path, 'rb') as f:
            labels = np.frombuffer(f.read(), np.uint8, offset=8)
    print("Done")
    
    return labels

def _load_img(file_name):
    file_path = dataset_dir + "/" + file_name
    
    print("Converting " + file_name + " to NumPy Array ...")    
    
    # 使用gzip.open函数，以二进制模式读取图像文件，作为对象 f 存储在RAM中。
    with gzip.open(file_path, 'rb') as f:
            data = np.frombuffer(f.read(), np.uint8, offset=16)
    # 通过整形，将图像转换为一维数组
    data = data.reshape(-1, img_size)    
    print("Done")
    
    return data

### 4. 转换 MNIST 数据集图像为 PKL

以二进制模式写入 (`wb`) 图像与标签为本地 PKL 文件。在这里可以看到，Python 有 pickle 这个便利的功能。这个功能可以将程序运行中的对象保存为文件。如果加载保存过的 pickle 文件，可以立刻复原之前程序运行中的对象。

In [4]:
def _convert_numpy():
    dataset = {}
    # 读取key_file字典中的train_img，作为图片训练集
    dataset['train_img'] =  _load_img(key_file['train_img'])  
    # 读取key_file字典中的train_label，作为标签训练集
    dataset['train_label'] = _load_label(key_file['train_label'])   
    # 读取key_file字典中的test_img，作为标签训练集
    dataset['test_img'] = _load_img(key_file['test_img'])     
    # 读取key_file字典中的test_label，作为标签训练集
    dataset['test_label'] = _load_label(key_file['test_label'])     
    
    return dataset

def init_mnist():
    dataset = _convert_numpy()
    print("Creating pickle file ...")
    
    # 使用 pickle 将对象 f 保存为本地文件 mnist.pkl，
    # 同时依然作为对象 f 等待后续读取加载加载。
    with open(save_file, 'wb') as f:
        pickle.dump(dataset, f, -1)
    print("转换成功！")

if __name__ == '__main__':
    init_mnist()

Converting ./data/lab6/train-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting ./data/lab6/train-labels-idx1-ubyte.gz to NumPy Array ...
Done
Converting ./data/lab6/t10k-images-idx3-ubyte.gz to NumPy Array ...
Done
Converting ./data/lab6/t10k-labels-idx1-ubyte.gz to NumPy Array ...
Done
Creating pickle file ...
转换成功！


在上面的操作中，我们用 `pickle.dump()` 读取了 gzip 解压出来的图像；并写入硬盘 —— 序列化。

![](./img/pickle_06.png)

在下面的操作中，我们将再次用 `pickle.load()` 从硬盘读取文件 —— 反序列化。整个环节实际上都是在操作 Python 的对象 **`f`**。

### 5. 定义加载 MNIST 函数

`load_mnist()` 函数以“(训练图像 ,训练标签 )，(测试图像，测试标签 )”的形式返回读入的 MNIST 数据，设置参数如下：

```python
load_mnist(normalize=True, flatten=True, one_hot_label=False)
``` 

- normalize：设置是否将输入图像标准化为 0.0～1.0 的值。如果将该参数设置为 False，则输入图像的像素会保持原来的 0～255。
- flatten：设置是否展开输入图像（变成一维数组）。如果将该参数设置为 False，则输入图像为 1 × 28 × 28 的三维数组；若设置为 True，则输入图像会保存为由 784 个元素构成的一维数组。
- one_hot_label：设置是否将标签保存为 one-hot 表示（one-hot representation）。
    - one-hot 表示是仅正确解标签为 1，其余皆为 0 的数组，如：`[0,0,1,0,0,0,0,0,0,0]`。
      - 当 one_hot_label 为 False 时，只是像 7、 2 这样简单保存正确解标签
      - 当 one_hot_label 为 True 时，标签则保存为 one-hot 表示。

另外，用于读入 MNIST 数据集的 `load_mnist()` 函数内部也使用了 pickle 功能（在第 2 次及以后读入时）。利用 pickle 功能，可以高效地完成 MNIST 数据的准备工作。

In [5]:
# 定义one-hot表示函数，但由于主函数 load_mnist() 设置了 one_hot_label=False，
# 因此此函数不会被执行，此段代码忽略
"""
def _change_one_hot_label(X):
    T = np.zeros((X.size, 10))
    for idx, row in enumerate(T):
        row[X[idx]] = 1
        
    return T    
"""

def load_mnist(normalize=True, flatten=True, one_hot_label=False):
    """读入MNIST数据集
    
    Parameters
    ----------
    normalize : 将图像的像素值正规化为0.0~1.0
    one_hot_label : 
        one_hot_label为True的情况下，标签作为one-hot数组返回
        one-hot数组是指[0,0,1,0,0,0,0,0,0,0]这样的数组
    flatten : 是否将图像展开为一维数组
    
    Returns
    -------
    (训练图像, 训练标签), (测试图像, 测试标签)
    """
    if not os.path.exists(save_file):
        init_mnist()
        
    with open(save_file, 'rb') as f:
        
        # 通过利用对象 f，使用 pickle 加载存储在本地的数据
        dataset = pickle.load(f)        
    
    # normalize=True，执行标准化
    if normalize:
        for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].astype(np.float32)
            
            # 由于图像的像素值是0-255，直接除以255，将像素值标准化为[0-1]区间内
            dataset[key] /= 255.0      
    
    # one_hot_label=False，不执行one-hot表示，此操作忽略
    if one_hot_label:
        
        # 引用 change_one_hot_label() 函数执行one-hot表示
        dataset['train_label'] = _change_one_hot_label(dataset['train_label']) 
        
        # 引用 change_one_hot_label() 函数执行one-hot表示
        dataset['test_label'] = _change_one_hot_label(dataset['test_label'])   
    
    # flatten=True，此操作忽略
    if not flatten:
         for key in ('train_img', 'test_img'):
            dataset[key] = dataset[key].reshape(-1, 1, 28, 28)

    return (dataset['train_img'], dataset['train_label']), (dataset['test_img'], 
                                                            dataset['test_label']) 

预处理在神经网络（深度学习）中非常实用，其有效性已在提高识别性能和学习的效率等众多实验中得到证明。在刚才的例子中，作为一种预处理，我们将各个像素值除以 255，进行了简单的标准化。实际上，很多预处理都会考虑到数据的整体分布。比如，利用数据整体的均值或标准差，移动数据，使数据整体以 0为中心分布，或者进行正规化，把数据的延展控制在一定范围内。除此之外，还有将数据整体的分布形状均匀化的方法，即数据白化（whitening）等。

### 6. 显示 MNIST 图像

现在，我们试着显示 MNIST 图像，同时也确认一下数据。图像的显示使用 PIL（Python Image Library）模块。这里需要注意的是， `flatten=True` 时读入的图像是以一列（一维）NumPy 数组的形式保存的。因此，显示图像时，需要把它变为原来的 28 像素 × 28 像素的形状。可以通过 `reshape()` 方法的参数指定期望的形状，更改 NumPy 数组的形状。执行下述代码后，训练图像的第一张就会显示出来：

In [6]:
# 导入Python Image Library模块
from PIL import Image

def img_show(img):
    
    # 把保存为NumPy数组的图像数据转换为PIL用的数据对象
    pil_img = Image.fromarray(np.uint8(img)) 
    pil_img.show()

# flatten=True，以一列（一维）NumPy数组的形式读入的图像
# normalize=False，不再执行标准化
(x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False)

# 选取第一张图片
img = x_train[0]   
# 选取第一张图片标签
label = t_train[0] 

# 输出第一章图片标签
# 5
print(label)  

# (784,)
print(img.shape)  
# 把图像的形状变为原来的尺寸
img = img.reshape(28, 28)  
# (28, 28)
print(img.shape)  

# 显示图片
img_show(img)

5
(784,)
(28, 28)


### 7. 神经网络推理

在本实验中，我们假设学习已经完成， 所以学习到的参数被保存下来，保存在 **`./model/sample_weight.pkl`** 文件中。因此，在推理阶段，我们直接加载这些已经学习到的参数。使用 `init_network()` 函数会读入保存在 pickle 文件 `sample_weight.pkl` 中的学习到的权重参数。这个文件中以字典变量的形式 (`network`) 保存了权重和偏置参数。

In [7]:
# 定义隐藏层激活函数，使用sigmoid
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# 定义输出层激活函数，使用softmax
def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    # 溢出对策
    x = x - np.max(x) 
    return np.exp(x) / np.sum(np.exp(x))

# 读取数据
def get_data():
    (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, 
                                                      flatten=True, one_hot_label=False)
    return x_test, t_test

# 读入保存在pickle文件 sample_weight.pkl中的学习到的权重参数
def init_network():
    # 以二进制模式读取
    with open(data_path+"model/sample_weight.pkl", 'rb') as f: 
        network = pickle.load(f)
    return network

# 从字典中分别读取各层的权重值与偏置值
def predict(network, x):
    W1, W2, W3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']

    a1 = np.dot(x, W1) + b1
    # 引用上面的激活函数sigmoid
    z1 = sigmoid(a1)          
    a2 = np.dot(z1, W2) + b2
    # 引用上面的激活函数sigmoid
    z2 = sigmoid(a2)          
    a3 = np.dot(z2, W3) + b3
    # 引用上面的激活函数softmax
    y = softmax(a3)          

    return y

### 8. 测试分类精确度 (Accuracy)

首先获得 MNIST 数据集，生成网络。接着，用 for 语句逐一取出保存在 x 中的图像数据 ——

- 用 `predict()` 函数进行分类。`predict()` 函数以 NumPy 数组的形式输出各个标签对应的概率。比如，输出 `[0.1, 0.3, 0.2, ..., 0.04]` 的数组，该数组表示 `0` 的概率为 0.1，`1` 的概率为 0.3，等等。然后，我们取出这个概率列表中的最大值的索引（第几个元素的概率最高），作为预测结果。
- 用 `np.argmax(x)` 函数取出数组中的最大值的索引， `np.argmax(x)` 将获取被赋给参数 x 的数组中的最大值元素的索引。
- 最后，比较神经网络所预测的答案和正确解标签，将回答正确的概率作为识别精度。

In [8]:
x, t = get_data()
network = init_network()

accuracy_cnt = 0
for i in range(len(x)):
    # 每次输入一张图片，下一个循环输入另一幅图片
    y = predict(network, x[i]) 
    
    # 获取概率最高的元素的索引，即最大概率，也就是神经网络的分类结果
    p= np.argmax(y)            
    
    # 将分类结果与t_test中的标签进行对比，如果一致，准确数+1
    if p == t[i]:             
        accuracy_cnt += 1     

# 输出精确度为0.9352
print("Accuracy:" + str(float(accuracy_cnt) / len(x))) 

Accuracy:0.9352


通过对输出求和得出总和为 `1`，可证输出层采用 **softmax** 作为激活函数，进行分类输出，其中概率最大的为分类结果。

In [9]:
round(sum(y))

1

我们通过上述结果来确认一下多维数组的对应维度的元素个数是否一致。

In [10]:
print(x.shape)
print(network['W1'].shape)
print(network['W2'].shape)
print(network['W3'].shape)
print(y.shape)

(10000, 784)
(784, 50)
(50, 100)
(100, 10)
(10,)


忽略偏置，如下图所示。可以发现，多维数组的对应维度的元素个数确实是一致的。此外，我们还可以确认最终的结果是输出了元素个数为 10 的一维数组。

![](./img/2_06.png)

输入一个由 784 个元素（原本是一个 28 × 28 的二维数组）构成的一维数组后，输出一个有 10 个元素的一维数组。注意，上图这是只输入一张图像数据时的处理流程，我们共有 10000 张测试样本，因此 x 的形状是 (`10000, 784`) —— 实际上是将 10000 张图片逐一输出神经网络。

### 9. 设置 Batch Size

通过设置 `batch_size` 变量，一次打包处理 100 张图像，进行推理预测，测试测试分类精确度。

- **`range()`** 函数的三个参数分别为循环的开始值、结束值、步长。这里指定每次输入神经网络的图片循环，从 0 开始，以训练集图像的，步长为 `batch_size` 即 100；
- **`argmax()`** 获取值最大的元素的索引。不过这里需要注意的是，我们给定了参数 `axis=1`。这指定了在 Y 的 100 × 10 的数组中，沿着第 1 维方向（以第 1 维为轴，显示每张图片在 10 个分类中，每个分类的概率）找到值最大的元素的索引。即：在 100 张图片中，找到每张图片 10 个 Y 输出中的最大值，即最大概率 —— 如下图：

<img src="./img/4_06.jpg" width="50%">

最后，比较一下以批为单位进行分类的结果和实际的答案。为此，需要在 NumPy 数组之间使用比较运算符（==）生成由 True/False 构成的布尔型数组，并计算  True 的个数。之后，用 True 的个数除以总数，得到精确度。

In [11]:
# 批处理量
batch_size = 100 
accuracy_cnt = 0

for i in range(0, len(x), batch_size):
    # 每次输入一个batch，下一个循环输入另外一个batch
    x_batch = x[i:i+batch_size] 
    
    # 根据输入的x_batch，获得最后神经网络的输出y_batch
    y_batch = predict(network, x_batch) 
    p = np.argmax(y_batch, axis=1)
    
    # 比较以批为单位进行分类的结果和实际的答案，每次对比100张
    accuracy_cnt += np.sum(p == t[i:i+batch_size])  
    
# 输出精确度为0.9352
print("Accuracy:" + str(float(accuracy_cnt) / len(x))) 

Accuracy:0.9352


通过对 `y_batch` 的 1 轴（axis=1）输出求和得出总和为 `1`，可证 `y_batch` 的 1 轴（axis=1）为每张图片的输出层 **softmax** 激活函数输出，进行分类输出，其中概率最大的为分类结果。

In [12]:
print(np.sum(y_batch, axis=1))

[1.         0.99999994 1.0000001  1.         1.         1.
 1.0000001  0.99999994 1.         0.9999999  0.99999994 1.0000001
 1.0000001  1.0000001  0.99999994 1.         1.         0.99999994
 1.         1.0000002  0.99999994 1.0000001  1.         0.9999999
 1.         1.         1.         1.         1.0000001  1.
 1.         0.99999994 1.0000001  1.         1.         1.
 1.         1.         1.0000001  1.0000001  1.         1.0000001
 1.0000001  1.         1.         1.         0.99999994 0.9999999
 0.99999994 1.         1.         1.         0.99999994 1.
 1.         1.         1.0000001  1.0000001  1.0000002  1.0000001
 1.0000001  1.         0.9999998  0.9999998  0.99999994 0.9999999
 0.99999994 0.99999994 1.0000001  1.0000001  1.         0.99999994
 1.         1.         1.         0.99999994 1.0000001  1.
 0.99999994 1.         1.0000001  1.         1.         1.0000001
 1.0000001  0.99999994 1.         1.         0.99999994 1.0000001
 1.         1.         1.         1.       

批处理的实际操作，是将 x 的形状改为 `100 × 784`，将 100 张图像打包作为输入数据。

In [13]:
print(x_batch.shape)
print(network['W1'].shape)
print(network['W2'].shape)
print(network['W3'].shape)
print(y_batch.shape)

(100, 784)
(784, 50)
(50, 100)
(100, 10)
(100, 10)


输入数据的形状为 100 × 784，输出数据的形状为 100 × 10。这表示输入的 100 张图像的结果被一次性输出了。

![](./img/3_06.png)

### 实验小结

在本实验中，你初步了解了Sample (样本)、Epoch、Batch Size (批处理量)、Iterations (迭代次数) 之间的关联，实现了解压缩 gz 文件，并通过 pickle 库将其转换为 Numpy 对象。之后，分别通过逐个图像推理与批量打包处理，分别测试分类精确度，并得到相同的分数。